# cadCAD Model Creation

This notebook shows how to build an actual cadCAD model (from the spec builder side) versus the other cadCAD notebook shows how cadCAD models could be handed off to a data scientist with no need to build them.

## Initialize Mathematical Specification

In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

from simulation import (params_base, state_base, compute_starting_total_length, check_d_probability, post_processing_function,
                        percent_ending_in_d_metric, average_d_count_metric, plot_length_single_simulation)


#from math_spec_mapping import load_from_json
sys.path.append(os.path.abspath('../..'))
from MSML.src.math_spec_mapping import (load_from_json)

from copy import deepcopy
from src import math_spec_json

ms = load_from_json(deepcopy(math_spec_json))

## Create a cadCAD Simulation

In [2]:
blocks = ["DUMMY Length-2 Boundary Wiring",
               "DUMMY Length-1 Boundary Wiring",
               "DUMMY Control Wiring"]

state_space, parameter_space, model = (ms.build_cadCAD(blocks,
                  state_preperation_functions=[compute_starting_total_length],
                  parameter_preperation_functions=[check_d_probability],)).values()

## Data Scientist Interface is the Simulation Object

### Single Experiment

In [3]:
state: state_space = {'Dummy': {'Words': '', 'Total Length': None},
                                   'Time': 0,
                                   'Simulation Log': []}
params: parameter_space = {'DUMMY D Probability': 0.5,
 'DUMMY Length Multiplier': 3,
 'FP DUMMY Length-1 DEF Control Action': 'DUMMY Length-1 DEF Equal Weight Option',
 'FP DUMMY Length-2 ABC Combo Boundary Action': 'DUMMY Length-2 ABC Equal Weight Option'}

experiment = model.create_experiment(state, params, record_trajectory=True, use_deepcopy=True)
experiment.run(5)
trajectories = experiment.trajectories
display(experiment.trajectories[-2:])

[{'Dummy': {'Words': 'AABDACCDBAADBBBE', 'Total Length': 48},
  'Time': 12,
  'Simulation Log': [{'Time': 1, 'Word': 'AA', 'Length (Multiplied)': 6},
   {'Time': 2, 'Word': 'AAB', 'Length (Multiplied)': 9},
   {'Time': 3, 'Word': 'AABD', 'Length (Multiplied)': 12},
   {'Time': 4, 'Word': 'AABDAC', 'Length (Multiplied)': 18},
   {'Time': 5, 'Word': 'AABDACC', 'Length (Multiplied)': 21},
   {'Time': 6, 'Word': 'AABDACCD', 'Length (Multiplied)': 24},
   {'Time': 7, 'Word': 'AABDACCDBA', 'Length (Multiplied)': 30},
   {'Time': 8, 'Word': 'AABDACCDBAA', 'Length (Multiplied)': 33},
   {'Time': 9, 'Word': 'AABDACCDBAAD', 'Length (Multiplied)': 36},
   {'Time': 10, 'Word': 'AABDACCDBAADBB', 'Length (Multiplied)': 42},
   {'Time': 11, 'Word': 'AABDACCDBAADBBB', 'Length (Multiplied)': 45},
   {'Time': 12, 'Word': 'AABDACCDBAADBBBE', 'Length (Multiplied)': 48}]},
 {'Dummy': {'Words': 'AABDACCDBAADBBBEBCBF', 'Total Length': 60},
  'Time': 15,
  'Simulation Log': [{'Time': 1, 'Word': 'AA', 'Length 

## Batch Experiments

In [6]:
from typing import List
state: list = [{'Dummy': {'Words': '', 'Total Length': None},
                                   'Time': 0,
                                   'Simulation Log': []},
                            {'Dummy': {'Words': '', 'Total Length': None},
                                   'Time': 0,
                                   'Simulation Log': []}]
params: list = [{'DUMMY D Probability': 0.5,
 'DUMMY Length Multiplier': 3,
 'FP DUMMY Length-1 DEF Control Action': 'DUMMY Length-1 DEF Equal Weight Option',
 'FP DUMMY Length-2 ABC Combo Boundary Action': 'DUMMY Length-2 ABC Equal Weight Option'},
 {'DUMMY D Probability': 0.1,
 'DUMMY Length Multiplier': 3,
 'FP DUMMY Length-1 DEF Control Action': 'DUMMY Length-1 DEF Equal Weight Option',
 'FP DUMMY Length-2 ABC Combo Boundary Action': 'DUMMY Length-2 ABC Equal Weight Option'}]
batch_experiments = model.create_batch_experiments(state, params, record_trajectory=True, use_deepcopy=True)
batch_experiments.run(3)
print(batch_experiments.trajectories)
print(len(batch_experiments.trajectories))
print(len(batch_experiments.trajectories[0]))

[[{'Dummy': {'Words': '', 'Total Length': 0}, 'Time': 0, 'Simulation Log': []}, {'Dummy': {'Words': 'BCCD', 'Total Length': 12}, 'Time': 3, 'Simulation Log': [{'Time': 1, 'Word': 'BC', 'Length (Multiplied)': 6}, {'Time': 2, 'Word': 'BCC', 'Length (Multiplied)': 9}, {'Time': 3, 'Word': 'BCCD', 'Length (Multiplied)': 12}]}, {'Dummy': {'Words': 'BCCDCBCD', 'Total Length': 24}, 'Time': 6, 'Simulation Log': [{'Time': 1, 'Word': 'BC', 'Length (Multiplied)': 6}, {'Time': 2, 'Word': 'BCC', 'Length (Multiplied)': 9}, {'Time': 3, 'Word': 'BCCD', 'Length (Multiplied)': 12}, {'Time': 4, 'Word': 'BCCDCB', 'Length (Multiplied)': 18}, {'Time': 5, 'Word': 'BCCDCBC', 'Length (Multiplied)': 21}, {'Time': 6, 'Word': 'BCCDCBCD', 'Length (Multiplied)': 24}]}, {'Dummy': {'Words': 'BCCDCBCDBAAF', 'Total Length': 36}, 'Time': 9, 'Simulation Log': [{'Time': 1, 'Word': 'BC', 'Length (Multiplied)': 6}, {'Time': 2, 'Word': 'BCC', 'Length (Multiplied)': 9}, {'Time': 3, 'Word': 'BCCD', 'Length (Multiplied)': 12}, {